In [1]:
import os
import pandas as pd
import json

In [3]:
bbs_df = pd.read_csv('dataset_bbs_embench.csv')

In [9]:
bb_1 = bbs_df[(bbs_df['binary'] == 'nbody') & (bbs_df['basic_block_address'] == '100cc')]

In [11]:
instructions = bb_1['instruction']

instructions

4            AUIPC gp, 9
5     ADDI, gp, gp, 1780
6     ADDI, a0, gp, 1864
7           AUIPC a2, 10
8     ADDI, a2, a2, -280
9         SUB a2, a2, a0
10     ADDI, a1, zero, 0
11          JAL ra, 8732
Name: instruction, dtype: object

In [16]:
def convert_to_ssa(instructions):
    ssa_result = []
    mapping = {}
    register_counter = 1

    for line in instructions:
        parts = line.split()
        operation = parts[0]
        args = parts[1:]
        
        updated_args = []
        for arg in args:
            if ',' in arg:
                arg = arg.strip(',')
            if arg in mapping:
                updated_args.append(mapping[arg])
            else:
                if arg.isalpha():  # Check if the argument is a register
                    mapping[arg] = f"r{register_counter}"
                    register_counter += 1
                    updated_args.append(mapping[arg])
                else:
                    updated_args.append(arg)
        
        if len(updated_args) > 0 and updated_args[0] not in mapping:
            mapping[updated_args[0]] = f"r{register_counter}"
            updated_args[0] = mapping[updated_args[0]]
            register_counter += 1
        elif len(updated_args) > 0:
            version_number = int(mapping[updated_args[0]][1:]) + 1
            mapping[updated_args[0]] = f"r{version_number}"
            updated_args[0] = mapping[updated_args[0]]
        
        ssa_line = f"{operation} " + ", ".join(updated_args)
        ssa_result.append(ssa_line)

    return ssa_result

In [24]:
convert_to_ssa(instructions)

['AUIPC r2, 9',
 'ADDI, r3, r1, 1780',
 'ADDI, r3, r1, 1864',
 'AUIPC r4, 10',
 'ADDI, r5, r4, -280',
 'SUB r6, r4, r3',
 'ADDI, r7, r6, 0',
 'JAL r9, 8732']

In [37]:
bb_1 = bbs_df[(bbs_df['binary'] == 'statemate') & (bbs_df['basic_block_address'] == '123f4')]
bb1_inst = bb_1['instruction']

ssa_1 = convert_to_ssa(bb1_inst)

ssa_1

['ADDI, r2, r1, 15', 'ADDI, r3, a0, 0', 'BGEU r4, a2, 52']

In [38]:
bb_2 = bbs_df[(bbs_df['binary'] == 'aha-mont64') & (bbs_df['basic_block_address'] == '10d60')]
bb2_inst = bb_2['instruction']

ssa_2 = convert_to_ssa(bb2_inst)

ssa_2

['ADDI, r2, r1, 15', 'ADDI, r3, a0, 0', 'BGEU r4, a2, 52']

In [ ]:
def compare_ssa_arrays(ssa_array1, ssa_array2):
    if len(ssa_array1) != len(ssa_array2):
        return False

    for instr1, instr2 in zip(ssa_array1, ssa_array2):
        if instr1 != instr2:
            return False

    return True

In [40]:
compare_ssa_arrays(ssa_1, ssa_2)

True